In [1]:
from PIL import Image
import numpy as np
import os

path='/home/tongwang/data/test_food_image_2/'
im = Image.open('/home/tongwang/data/test_food_image_2/101&Burger_Sandwich$12.bmp')
im_array = np.array(im)
print(im_array.shape)

rows=im_array.shape[0]
cols=im_array.shape[1]
channels=3
train = 3500

In [33]:
pic_names = os.listdir(path)
pics = []
labels = []
names = []
for name in pic_names:
    im = Image.open(path+name)
    im_array = np.array(im)
    if len(im_array.shape) == 3:
        pics.append(im_array)
        label = name[name.index('&')+1:name.index('$')]
        names.append(name)
        if label == 'Burger_Sandwich':
            labels.append(0)
        elif label == 'Pizza':
            labels.append(1)
        else:
            labels.append(2)



# np.random.seed(10)
# np.random.shuffle(X)
# np.random.seed(10)
# np.random.shuffle(y)

X = np.array(pics) - 127
y = np.array(labels)
print(X.shape)
print(y.shape)

(3886, 64, 64, 3)
(3886,)


In [58]:
i = 99
print(names[i])
print(y[i])
img = Image.fromarray(X[i], 'RGB')
img.show()

369&Pizza$8.bmp
1


In [35]:
X_train = X[:train]
y_train = y[:train]
X_test = X[train:]
y_test = y[train:]
y_test

array([2, 0, 0, 2, 2, 2, 2, 1, 0, 2, 0, 0, 2, 2, 0, 0, 2, 1, 0, 1, 1, 0, 1,
       0, 1, 0, 1, 1, 1, 1, 0, 0, 2, 1, 2, 1, 1, 1, 2, 2, 1, 2, 2, 0, 0, 0,
       0, 0, 1, 0, 1, 2, 1, 1, 1, 1, 2, 0, 0, 0, 1, 0, 1, 2, 2, 0, 0, 2, 2,
       0, 1, 2, 2, 1, 2, 1, 0, 1, 1, 2, 0, 2, 0, 2, 0, 1, 2, 1, 0, 1, 1, 0,
       0, 2, 1, 0, 0, 2, 0, 1, 0, 1, 0, 2, 2, 2, 2, 0, 0, 0, 1, 2, 2, 2, 1,
       1, 0, 2, 0, 2, 1, 2, 1, 0, 2, 0, 0, 2, 0, 0, 0, 0, 0, 2, 2, 1, 0, 0,
       0, 0, 1, 0, 2, 1, 2, 1, 2, 1, 2, 0, 2, 0, 0, 2, 1, 0, 1, 2, 1, 0, 0,
       1, 0, 1, 0, 1, 2, 2, 0, 0, 1, 1, 2, 0, 2, 1, 2, 2, 1, 1, 1, 1, 0, 2,
       0, 2, 2, 0, 0, 1, 1, 0, 0, 0, 1, 2, 2, 1, 0, 2, 2, 1, 0, 0, 2, 1, 2,
       1, 1, 1, 2, 2, 1, 0, 0, 1, 2, 1, 0, 1, 1, 0, 0, 2, 0, 2, 1, 0, 1, 1,
       2, 2, 1, 1, 1, 2, 1, 0, 2, 0, 2, 2, 1, 2, 1, 2, 0, 0, 1, 2, 0, 1, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 1, 2, 2, 0, 0, 1, 0, 1, 0, 1, 0, 2, 1, 1, 2,
       0, 1, 1, 0, 2, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 2, 1, 2, 1, 0, 0, 2, 0,
       0, 2,

In [36]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
from keras.optimizers import SGD, RMSprop, Adam

In [37]:
Y_train = np_utils.to_categorical(y_train, 3)
Y_test = np_utils.to_categorical(y_test, 3)

In [69]:
batch_size = 32
nb_classes = 3
nb_epoch = 100
# sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)


min_loss = 1000000
model = Sequential()
model.add(Convolution2D(32, 5, 5,border_mode='valid', subsample=(1,1),
                        input_shape=(rows, cols,channels), dim_ordering='tf'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2),dim_ordering='tf'))
model.add(Convolution2D(32, 3, 3,border_mode='valid', subsample=(1,1),dim_ordering='tf'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2),dim_ordering='tf'))
model.add(Convolution2D(16, 3, 3,border_mode='valid', dim_ordering='tf',subsample=(1,1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2),dim_ordering='tf'))
model.add(Dropout(.5))
model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))
for e in range(epoch):
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['accuracy'])

hist=model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch,
          verbose=1,validation_split=0.1, shuffle=True)
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Train on 3150 samples, validate on 350 samples
Epoch 1/100
3150/3150 [==============================] - 3s - loss: 1.2896 - acc: 0.3156 - val_loss: 1.0988 - val_acc: 0.3343
Epoch 2/100
3150/3150 [==============================] - 3s - loss: 1.0995 - acc: 0.3333 - val_loss: 1.0986 - val_acc: 0.2771
Epoch 3/100
3150/3150 [==============================] - 3s - loss: 1.1445 - acc: 0.3356 - val_loss: 1.1000 - val_acc: 0.2771
Epoch 4/100
3150/3150 [==============================] - 3s - loss: 1.1033 - acc: 0.3317 - val_loss: 1.0993 - val_acc: 0.2800
Epoch 5/100
3150/3150 [==============================] - 3s - loss: 1.0990 - acc: 0.3251 - val_loss: 1.0983 - val_acc: 0.3914
Epoch 6/100
3150/3150 [==============================] - 3s - loss: 1.0992 - acc: 0.3337 - val_loss: 1.0980 - val_acc: 0.3914
Epoch 7/100
3150/3150 [==============================] - 3s - loss: 1.1024 - acc: 0.3365 - val_loss: 1.0984 - val_acc: 0.3914
Epoch 8/100
3150/3150 [==============================] - 3s - loss: 1.1

In [49]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_43 (Convolution2D) (None, 62, 62, 32)    896         convolution2d_input_15[0][0]     
____________________________________________________________________________________________________
activation_88 (Activation)       (None, 62, 62, 32)    0           convolution2d_43[0][0]           
____________________________________________________________________________________________________
maxpooling2d_43 (MaxPooling2D)   (None, 31, 31, 32)    0           activation_88[0][0]              
____________________________________________________________________________________________________
convolution2d_44 (Convolution2D) (None, 29, 29, 32)    9248        maxpooling2d_43[0][0]            
___________________________________________________________________________________________

In [25]:
import cv2
winSize = (64,64)
blockSize = (16,16)
blockStride = (8,8)
cellSize = (8,8)
nbins = 9
derivAperture = 1
winSigma = 4.
histogramNormType = 0
L2HysThreshold = 2.0000000000000001e-01
gammaCorrection = 0
nlevels = 64
hog = cv2.HOGDescriptor(winSize,blockSize,blockStride,cellSize,nbins,derivAperture,winSigma,
                        histogramNormType,L2HysThreshold,gammaCorrection,nlevels)
winStride = (8,8)
padding = (8,8)
locations = ((10,20),)
im = cv2.imread('/home/tongwang/data/test_food_image_2/101&Fast_Food$12.bmp')
h = hog.compute(im)
print(h)

[[ 0.46322173]
 [ 0.        ]
 [ 0.        ]
 ..., 
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]]


In [52]:
X_train2 = X_train.reshape(X_train.shape[0], -1)
X_test2 = X_test.reshape(X_test.shape[0], -1)

In [68]:
sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model = Sequential()
model.add(Dense(30,input_shape=(X_train2.shape[1],)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['accuracy'])

model.fit(X_train2, Y_train, batch_size=64, nb_epoch=300,
          verbose=1, validation_data=(X_test2, Y_test))
score = model.evaluate(X_test2, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Train on 3500 samples, validate on 386 samples
Epoch 1/300
3500/3500 [==============================] - 0s - loss: 10.6917 - acc: 0.3351 - val_loss: 11.4413 - val_acc: 0.2902
Epoch 2/300
3500/3500 [==============================] - 0s - loss: 10.6456 - acc: 0.3377 - val_loss: 11.4413 - val_acc: 0.2902
Epoch 3/300
3500/3500 [==============================] - 0s - loss: 10.7767 - acc: 0.3260 - val_loss: 11.4413 - val_acc: 0.2902
Epoch 4/300
3500/3500 [==============================] - 0s - loss: 10.5791 - acc: 0.3380 - val_loss: 11.4413 - val_acc: 0.2902
Epoch 5/300
3500/3500 [==============================] - 0s - loss: 10.7538 - acc: 0.3240 - val_loss: 11.4413 - val_acc: 0.2902
Epoch 6/300
3500/3500 [==============================] - 0s - loss: 10.2682 - acc: 0.3503 - val_loss: 11.4413 - val_acc: 0.2902
Epoch 7/300
3500/3500 [==============================] - 0s - loss: 10.5354 - acc: 0.3371 - val_loss: 11.4413 - val_acc: 0.2902
Epoch 8/300
3500/3500 [==============================] - 

In [66]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_82 (Dense)                 (None, 50)            614450      dense_input_7[0][0]              
____________________________________________________________________________________________________
activation_145 (Activation)      (None, 50)            0           dense_82[0][0]                   
____________________________________________________________________________________________________
dropout_68 (Dropout)             (None, 50)            0           activation_145[0][0]             
____________________________________________________________________________________________________
dense_83 (Dense)                 (None, 100)           5100        dropout_68[0][0]                 
___________________________________________________________________________________________